# Goal

This notebooks shows how to execute queries and stored procedures using `PIVOT/utils/sql_utils/`.

## 0. Set Up
---
To begin, we'll need to import all necessary modules. This should come installed with the virtual environment provided by [`environment.yml`](../environment.yml).

If not, please install the modules with the following commands:

```bash
pip install <module_name>
```

or 

```bash
conda install <module_name>
```

Note that we have a major dependencies on `pymssql`, which is a Python wrapper for SQL integration. More detailts can be found [here](https://pymssql.readthedocs.io/en/stable/index.html). Another option is the newer [PYODBC](https://learn.microsoft.com/en-us/sql/connect/python/pyodbc/python-sql-driver-pyodbc?view=sql-server-ver16), however, we picked PYMSSQL because it doesn't required additional installations of system-specific drivers. However, PYMSSQL is now maintained by a volunteer network of developers. So, bugs may take longer to be resolved.

Additionally, you must have the correct authentication keys for Azure SQL in `./PIVOT/config/config.yaml`!

You can ignore the safely ignore the warning `WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager` as this is something only for when Streamlit is running.


In [2]:
import sys
# import sql_utils from the PIVOT app
sys.path.append('../PIVOT/')
import utils.sql_utils as sq
from importlib import reload
reload(sq)

2024-03-13 12:29:51.341 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-03-13 12:29:51.344 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


<module 'utils.sql_utils' from 'C:\\Users\\ysman\\Desktop\\Capstone\\PIVOT\\notebooks\\../PIVOT\\utils\\sql_utils.py'>

## 1. Example Commands

To start, we can run any given SQL query with the `sq.run_sql_query()`. 

NOTE: If no queries have been run in the past 24 hours, the Azure SQL might become dormant and take a while to load. If this is the case, then any query will output the error:

```
OperationalError: (40613, b'DB-Lib error message 20018, severity 20:\nGeneral SQL Server error: Check messages from the SQL Server\nDB-Lib error message 20002, severity 9:\nAdaptive Server connection failed (capstoneservercjault.database.windows.net)\nDB-Lib error message 20002, severity 9:\nAdaptive Server connection failed (capstoneservercjault.database.windows.net)\n')
```

If you encounter this, **please rerun the command after waiting 30 seconds** until the system turns on. As such, it is recommended that before doing any complex queries, please run a simple query such as the following to check whether the system is active.

The following command find the count of all labels so far.

In [6]:
sq.run_sql_query("Select count(*) from Labels;")

,
0,399998


This next query shows how to gather table metadata for the `Predictions` table.

In [8]:
sq.run_sql_query(
"""
SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE, COLUMN_DEFAULT
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'predictions';
"""
)

,COLUMN_NAME,DATA_TYPE,IS_NULLABLE,COLUMN_DEFAULT
0,m_id,int,NO,None
1,i_id,int,NO,None
2,class_prob,varchar,YES,None
3,pred_label,varchar,YES,None


## 2. Stored Procedures
---
Stored procedures are precompiled SQL queries or statements that are stored in a database. They allow you to group one or more SQL statements into a single unit and execute them whenever needed by calling the procedure rather than rewriting the same SQL code multiple times.

In our case, stored procedures offer several advantages:

**Modularity**: They promote modular programming practices by encapsulating SQL code into reusable units.

**Performance**: Since stored procedures are precompiled and stored in the database, they can enhance performance by reducing the overhead of parsing and compiling SQL statements each time they are executed.

**Consistency and Maintainability**: They promote consistency in database operations by ensuring that the same logic is applied consistently across different applications. Additionally, if changes are needed, you only need to modify the stored procedure rather than updating the SQL code in multiple places.

**Security**: Stored procedures can help improve security by controlling access to data through a well-defined interface. Users can be granted permissions to execute stored procedures without giving them direct access to underlying tables. Note, we HAVE NOT set up this functionality but it is something to look into.

**Reduced Network Traffic**: By executing complex operations on the database server, stored procedures can reduce the amount of data transferred between the database server and client applications, thereby minimizing network traffic.


To get a list of all stored procedures in the system, please run the following query:

In [13]:
check_procedures = """
    SELECT 
        name AS 'ProcedureName',
        create_date AS 'CreateDate',
        modify_date AS 'ModifyDate'
    FROM 
        sys.procedures;
"""
sq.run_sql_query(check_procedures)

,ProcedureName,CreateDate,ModifyDate
0,AL_RANKINGS,2024-01-28 03:51:35.657,2024-02-28 03:44:22.757
1,GENERATE_RANDOM_TEST_SET,2024-01-28 09:49:52.003,2024-02-05 09:17:59.247
2,MODEL_EVALUATION_MAX_CONSENSUS_FILTERING,2024-01-28 10:40:44.150,2024-02-28 03:47:02.803
3,MODEL_EVALUATION_NON_TEST,2024-01-28 18:34:04.107,2024-01-28 18:34:04.107
4,AL_TRAIN_SET,2024-01-29 02:09:27.330,2024-01-29 02:09:27.330
5,GENERATE_IMAGES_TO_PREDICT,2024-02-06 00:30:00.193,2024-02-06 00:30:00.193
6,GENERATE_IMAGES_TO_METRIZE,2024-02-06 00:30:25.440,2024-02-06 00:30:25.440


We've wrote several stored procedures in `../PIVOT/utils/stored_procedures`. Specifically:

* `AL_RANKINGS`: Stored at [Labeling_Ranking.sql](../PIVOT/utils/stored_procedures/Labeling_Ranking.sql), this stored procedure returns sets of image metadata for either evaluation or retraining purpose using random sample or a known, precomputed active learning (AL) method. 

* `MODEL_EVALUATION_NON_TEST`: Stored at [Model_Evaluation_NonTest.sql](../PIVOT/utils/stored_procedures/Model_Evaluation_NonTest.sql), this returns the image metadata for all non-test image data.

* `MODEL_EVALUATION_MAX_CONSENSUS_FILTERING`: Stored at [Model_Evaluation_Filtering.sql](../PIVOT/utils/stored_procedures/Model_Evaluation_Filtering.sql), this returns the image metadata for all test image data.

* `AL_TRAIN_SET`: Stored at [Model_Training.sql](../PIVOT/utils/stored_procedures/Model_Training.sql), this returns all the image metadata for images to use for training.

* `GENERATE_RANDOM_TEST_SET`: Stored at [Generate_Random_Test_Set.sql](../PIVOT/utils/stored_procedures/Generate_Random_Test_Set.sql), called only once, this generate an initial set of test data from the 5M images in BLOB.

* `GENERATE_IMAGES_TO_PREDICT`: Stored at [Images_To_Predict.sql](../PIVOT/utils/stored_procedures/Images_To_Predict.sql), this generates image metadata for images that are in the `Images` table but haven't had model predictions.

* `GENERATE_IMAGES_TO_METRIZE`: Stored at [Images_To_Metrize.sql](../PIVOT/utils/stored_procedures/Images_To_Metrize.sql), this generate image metadata for images that haven't had AL metrics like entropy calculated for them.


Note that a majority of these stored procedures have prebuilt wrapper functions that take in a set # of arguments and should only be called in that manner. For example, getting the images to label should be called with the `get_label_rank_df()` function, which internally calls the `AL_RANKINGS` stored procedure (SP).
Additionally:
* `get_images_to_metrize()` get new images for metric calculation based on model and dissimilarity ID and calls the `GENERATE_IMAGES_TO_METRIZE` stored prodecure.
* `get_images_to_predict()` get new images for model predictions based on model ID and calls the `GENERATE_IMAGES_TO_PREDICT` stored procedure.
* `generate_random_evaluation_set()` generate a random evaluation set for model testing and calls the `GENERATE_RANDOM_TEST_SET` stored procedure.
* `get_test_set_df()` get labeled test data and predictions for model evaluation and calls the `MODEL_EVALUATION_MAX_CONSENSUS_FILTERING` stored procedure.
* `get_train_df()`: Get a DataFrame for model training based on model, dissimilarity ID, and class labels and calls the `AL_TRAIN_SET` stored procedure.

In [14]:
df = sq.get_label_rank_df(model_id=1,dissimilarity_id=1,batch_size=5,)

In [15]:
df.head()

,IMAGE_ID,BLOB_FILEPATH,UNCERTAINTY,PRED_LABEL,PROBS,RANK_SCORE
0,1301298,NAAMES_ml/D20160602T164019_IFCB107/IFCB107D201...,2.067445,Other,"{'Chloro': 0.016140414, 'Cilliate': 0.00167912...",2.067445
1,638839,NAAMES_ml/D20170829T221958_IFCB107/IFCB107D201...,1.952166,Crypto,"{'Chloro': 0.051081363, 'Cilliate': 0.00090885...",1.952166
2,1073933,NAAMES_ml/D20170920T174345_IFCB107/IFCB107D201...,1.918797,Cilliate,"{'Chloro': 0.22434974, 'Cilliate': 0.2447636, ...",1.918797
0,400252,NAAMES_ml/D20160524T225721_IFCB107/IFCB107D201...,1.000000,Unidentifiable,"{'Chloro': 0.0001842739, 'Cilliate': 3.9586052...",1.000000
1,400268,NAAMES_ml/D20160524T225721_IFCB107/IFCB107D201...,1.000000,Other,"{'Chloro': 0.024367146, 'Cilliate': 4.281957e-...",1.000000


In [6]:
sq.get_test_set_df(model_id=1, minimum_percent=0.0, sp_name='MODEL_EVALUATION_NON_TEST')

,IMAGE_ID,PRED_LABEL,CONSENSUS
0,3,Confirmed Label A,Confirmed Label A


If you need to update any stored procedures, please modify the source file and run the following command: `create_alter_stored_procedure()`.

In [18]:
sq.create_alter_stored_procedure('AL_TRAIN_SET')

Using preset file to create procedure AL_TRAIN_SET: C:\Users\ysman\Desktop\Capstone\PIVOT\PIVOT\utils\stored_procedures\Model_Training.sql


If you need to add a new stored procedure, please save the file in `../PIVOT/utils/stored_procedures/` and update the relevant file paths and argument types inside the `../PIVOT/utils/sql_constants.py`.

### NOTE: Do not delete any records from the labels using `run_sql_query`. Instead, use the `delete_labels_cleanup()` function.

This is because each time the a new label is inserted, the corresponding metric is decayed by a set constant. So, if the label is removed, it needs to revert the metric to the point prior to insertion.